In [232]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/andrew/src/otp-scheduler


In [233]:
import pandas as pd
from src import parse

In [234]:
def sort_schedule(df):
    return df.sort_values(by=['zone', 'priority_rank', 'code']).reset_index(drop=True)

In [235]:
# ASSUMPTIONS
MEAL_CAPACITY = 20000
MAX_DROPOFFS_PER_RUN = 4
MIN_MEALS_PER_RUN = 50

In [236]:
restaurants = parse.parse_from_restaurants_csv('../../examples/boston/20200415/restaurants.csv').set_index('code')

In [237]:
restaurants.shape

(12, 23)

In [238]:
hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200415/hospitals.csv'))
schedule = hospitals.iloc[:, 5:]

In [239]:
vip_hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200415/vips.csv'))
vip_schedule = vip_hospitals.iloc[:, 5:]
vip_schedule.shape

(34, 21)

In [240]:
# results = sort_schedule(parse.parse_from_results_csv('../../examples/boston/20200405/results.csv'))
results = sort_schedule(parse.parse_from_results_csv('../../output.csv'))
results.shape

(34, 26)

In [241]:
results_schedule = results.iloc[:, 5:]
results_schedule_columns = results_schedule.columns

In [242]:
log = []
for i, row in results_schedule.iterrows():
    for j, val in enumerate(row):
        if not pd.isnull(val):
            quantity = schedule.iloc[i, j]
            vip = False
            if pd.isnull(quantity) and not pd.isnull(vip_schedule.iloc[i, j]):
                quantity = vip_schedule.iloc[i, j]
                vip = True
            log.append([
                hospitals.iloc[i, 1], 
                hospitals.iloc[i, 4],
                results_schedule_columns[j], 
                val, 
                quantity, 
                restaurants[results_schedule_columns[j]][val],
                vip,
            ])

In [243]:
meal_log = pd.DataFrame(log, columns=['hospital', 'zone', 'meal', 'restaurant', 'quantity', 'restaurant_capacity', 'vip'])
meal_log

,hospital,zone,meal,restaurant,quantity,restaurant_capacity,vip
0,BIDMC - West Campus,1,lunch_mon,SU,45.0,400.0,False
1,BIDMC - West Campus,1,lunch_tues,SU,45.0,400.0,False
2,BIDMC - West Campus,1,lunch_wed,PG,45.0,700.0,False
3,BIDMC - West Campus,1,lunch_thurs,SU,45.0,400.0,False
4,BIDMC - West Campus,1,lunch_fri,SU,45.0,400.0,False
...,...,...,...,...,...,...,...
236,Newton Wellesley Hospital,4,dinner_wed,BB,50.0,400.0,False
237,Newton Wellesley Hospital,4,dinner_thurs,BB,50.0,400.0,False
238,Newton Wellesley Hospital,4,dinner_fri,SH,50.0,300.0,False
239,Newton Wellesley Hospital,4,dinner_sat,MD,50.0,400.0,False


In [244]:
# Ensure all VIP orders are fulfilled
vip_schedule_dense = vip_schedule.dropna(how='all', axis=0).dropna(how='all', axis=1)
vip_schedule_dense
vip_orders = []
missing_meals = []
for i, row in vip_schedule_dense.iterrows():
    for j, val in enumerate(row):
        if not pd.isnull(val):
            order = meal_log[(meal_log['hospital'] == hospitals.iloc[i, 1]) & (meal_log['meal'] == vip_schedule_dense.columns[j])]
            vip_orders.append(order)
            if order.empty:
                missing_meals.append((hospitals.iloc[i, 1], vip_schedule_dense.columns[j]))
                
test(len(missing_meals) == 0, 'Some VIP meal orders have not been fulfilled', missing_meals)

In [245]:
vip_orders_df = pd.concat(vip_orders).reset_index()

## Rules / Validation Checks:
High-level
- [x] Within budget (# meals < capacity)

Restaurants
- [x] Restaurant has capacity for that order
- [x] Restaurant is delivering to one cluster per meal time
- [x] Restaurant is delivering at least 50 meals for a meal time they are assigned
- [x] Restaurant is not delivering to more than 4 dropoffs per meal time
- [x] Restaurant is within their max days per week

Allocation
- [x] VIP Orders are all fulfilled
- [x] All orders are fully-served (this is pretty much intrinsically assumed, given the format of our data)
- [x] Every HP has > 0 orders

In [246]:
def test(condition, *print_args):
    if not condition:
        print('TEST FAILURE:', print_args[0])
        for arg in print_args[1:]:
            print(arg)

In [247]:
# NOTHING in here should be null! This means something isn't matching up with our priors in our spreadsheets!
test(meal_log.isnull().sum().sum() == 0, 'we have nulls in our dense meal log set!', meal_log.isnull().sum().sum())

In [248]:
# Within budget
test(meal_log['quantity'].sum() < MEAL_CAPACITY, 'we are above meal capacity', meal_log['quantity'].sum(), MEAL_CAPACITY)

In [249]:
# Restauarant assertions
under_capacity = meal_log[meal_log['restaurant_capacity'] < meal_log['quantity']]
test(under_capacity.empty, 'Some restaurants are assinged a meal that is over their capacity')

In [250]:
# Restaurant is delivering one cluster per meal time
restaurant_delivery_zones = meal_log.groupby(['meal', 'restaurant'])[['zone']].nunique()
multiple_zones = restaurant_delivery_zones[restaurant_delivery_zones['zone'] > 1]
test(multiple_zones.empty, 'Some restaurants are delivering to more than one zone for a meal!', multiple_zones)

In [251]:
# Restaurant is delivering at least `MIN_MEALS_PER_RUN` per run
restaurant_quantity_per_meal = meal_log.groupby(['meal', 'restaurant'])[['quantity']].sum()
small_orders_per_meal = restaurant_quantity_per_meal[restaurant_quantity_per_meal['quantity'] < MIN_MEALS_PER_RUN]
test(small_orders_per_meal.empty, 'Some restaurants are delivering a low quantity of meals for an order!', small_orders_per_meal)

TEST FAILURE: Some restaurants are delivering a low quantity of meals for an order!
                            quantity
meal            restaurant          
breakfast_fri   BB              30.0
breakfast_mon   SH              12.0
breakfast_sun   BB              20.0
                PG              12.0
breakfast_thurs SH              12.0
breakfast_tues  BB              30.0
                PG              12.0
breakfast_wed   SH              12.0
dinner_fri      BB              15.0
dinner_sat      LD              45.0
dinner_sun      BB              15.0
lunch_fri       IG              18.0
lunch_sat       MD              30.0
lunch_thurs     IG              18.0
lunch_tues      IG              18.0


In [252]:
# Restaurant is delivering no more than `MAX_DROPOFFS_PER_RUN` dropoffs per run
dropoffs_per_run = meal_log.groupby(['meal', 'zone', 'restaurant'])[['hospital']].count()
too_many_dropoffs = dropoffs_per_run[dropoffs_per_run['hospital'] > MAX_DROPOFFS_PER_RUN]
test(too_many_dropoffs.empty, 'Some restaurants are delivering to too many dropoff points per run!', too_many_dropoffs)

In [253]:
# Restaurant is within their max days per week

# Roll up all the meals into days
def column_to_day(series):
    days = set(series.map(lambda m: m.replace('lunch', '').replace('breakfast', '').replace('dinner', '').replace('_', '')))
    return len(days)
    

days_worked = meal_log.groupby(['restaurant']).agg({'meal': [('Days Worked', column_to_day)]}).reset_index()
days_worked.columns = ['restaurant', 'Days Worked']
days_with_capacity = days_worked.merge(restaurants[['max_days_per_week']], left_on='restaurant', right_on='code')
days_over_capacity = days_with_capacity[days_with_capacity['Days Worked'] > days_with_capacity['max_days_per_week']]
test(too_many_dropoffs.empty, 'Some restaurants are working more days than requested this week', days_over_capacity)

In [254]:
hospital_meals_rollup = meal_log[['hospital', 'quantity']].groupby('hospital').agg({
    'quantity': [('total_meals_received', 'sum'), ('total_orders_fulfilled', 'count')]
}).sort_values([('quantity', 'total_meals_received')], ascending=False).reset_index()
hospital_meals_rollup.columns = ['hospital', 'total_meals_received', 'total_orders_fulfilled']

In [255]:
vip_orders_rollup = vip_orders_df.groupby('hospital')[['index']].count().reset_index()
vip_orders_rollup.columns = ['hospital', 'vip_meals_count']
vip_orders_rollup

,hospital,vip_meals_count
0,Boston Healthcare for the Homeless,14
1,Boston Medical Center,7
2,Brigham and Women's - Faulkner ED,4
3,Brigham and Women's Hospital - ED,4
4,Harvard Street Neighborhood Health Center,5
5,Massachusetts General Hospital - PICU,10


In [256]:
hospital_meals_rollup = meal_log[['hospital', 'quantity']].groupby('hospital').agg({
    'quantity': [('total_meals_received', 'sum'), ('total_orders_fulfilled', 'count')],
}).sort_values([('quantity', 'total_meals_received')], ascending=False).reset_index()
hospital_meals_rollup.columns = ['hospital', 'total_meals_received', 'total_orders_fulfilled']

hospital_meals_rollup = hospital_meals_rollup.merge(vip_orders_rollup, on='hospital', how='left').fillna(0)

In [257]:
# Every HP has > 0 orders
hospitals_no_orders = hospital_meals_rollup[hospital_meals_rollup['total_orders_fulfilled'] == 0]
test(hospitals_no_orders.empty, 'Some Hospitals received no orders!', hospitals_no_orders)

## KPIs:
- [X] Meals scheduled for delivery
- [x] Utilization % (number of meals delivered / capacity)
- [x] Meals per Delivery Run
- [x] Total meals and number of orders for each HP
- [x] Total meals and number of orders for each Restaurant
- [x] Equitable meals served: all HPs are within 1 order of each other, excluding VIP orders

In [258]:
# Number of meals delivered
meals_scheduled = meal_log['quantity'].sum()
meals_scheduled

17872.0

In [259]:
# Meal Utilization %
utilization = meals_scheduled / MEAL_CAPACITY
print(str(utilization * 100) + '%')

89.36%


In [260]:
grouped_by_meal_restaurant = meal_log.groupby(['meal', 'restaurant']).agg({
    'quantity': ['sum', 'count']
}).sort_values(by='meal', ascending=False)
# grouped_by_meal_restaurant.head(10)

In [261]:
# Meals delivered by Run, and Number of Dropoff Points
# CAVEAT: this ASSUMES that a restaurant is delivering to a single cluster for one meal time. Careful!
meals_per_run_agg = grouped_by_meal_restaurant.agg(['median', 'mean', 'min', 'max'])
meals_per_run_agg.columns = ['Meals per Meal Time', 'Dropoff Points']
meals_per_run_agg

,Meals per Meal Time,Dropoff Points
median,149.000000,2.000000
mean,161.009009,2.171171
min,12.000000,1.000000
max,540.000000,4.000000


In [262]:
# Total number of meals delivered by restaurant
meal_log.groupby('restaurant')[['quantity']].sum().sort_values(by='quantity', ascending=False)

,quantity
restaurant,
PG,3523.0
SU,2873.0
LD,2650.0
MD,2095.0
BB,1980.0
SH,1739.0
FC,1630.0
PO,995.0
IG,299.0


In [263]:
# Total meals and orders for each Restaurant
meal_log.groupby('restaurant').agg({
    'quantity': [('meal_sum', 'sum'), ('orders_fulfilled', 'count')]
}).sort_values([('quantity', 'meal_sum')], ascending=False)

quantity                 
           meal_sum orders_fulfilled
restaurant                          
PG           3523.0               38
SU           2873.0               43
LD           2650.0               35
MD           2095.0               35
BB           1980.0               29
SH           1739.0               20
FC           1630.0               14
PO            995.0               18
IG            299.0                7
ME             88.0                2

In [264]:
# Total numbers of meals and orders received for each HP
# NB: this variable is defined above, as it's used in a validation test
hospital_meals_rollup

,hospital,total_meals_received,total_orders_fulfilled,vip_meals_count
0,Brigham & Women's Hospital (COVID ICUs),3510.0,17,0.0
1,Boston Medical Center,2600.0,9,7.0
2,Massachusetts General Hospital - Central,1100.0,3,0.0
3,Cambridge Health Alliance - Everett Hospital,900.0,7,0.0
4,Cambridge Health Alliance - Cambridge Hospital,805.0,7,0.0
5,Whittier Street Health Center,750.0,5,0.0
6,Newton Wellesley Hospital,700.0,14,0.0
7,Carney Hospital - ED,700.0,7,0.0
8,Codman Square Health Center,700.0,5,0.0
9,VA Medical Center - West Roxbury,480.0,6,0.0


## Hospital Equity Check

In [265]:
# Hospital Equity check
max_row = hospital_meals_rollup[
    hospital_meals_rollup['total_orders_fulfilled'] == hospital_meals_rollup['total_orders_fulfilled'
].max()]
min_row = hospital_meals_rollup[
    hospital_meals_rollup['total_orders_fulfilled'] == hospital_meals_rollup['total_orders_fulfilled'
].min()]

order_range = (max_row['total_orders_fulfilled'].values[0] - min_row['total_orders_fulfilled'].values[0])
print('INCLUDING VIP MEALS')
print('RANGE OF ORDERS FOR HOSPITALS:', order_range)
if order_range > 1:
    print('This range should ideally only be 1, once we remove VIP orders!')
print('Note: this is inclusive of VIP orders. Please verify this discrepancy!')
print('------------------------------')
print(
    'MAXIMUM Order for Hospital:', 
    max_row['total_orders_fulfilled'].values[0], 
    'for Hospital(s):', 
)
for _, row in max_row.iterrows():
    print('-', row['hospital'], '\tNumber of VIP orders:', row['vip_meals_count'])
print()
print(
    'MINIMUM Order for Hospital:', 
    min_row['total_orders_fulfilled'].values[0], 
    'for Hospital(s):', 
)
for _, row in min_row.iterrows():
    print('-', row['hospital'], '\tNumber of VIP orders:', row['vip_meals_count'])


INCLUDING VIP MEALS
RANGE OF ORDERS FOR HOSPITALS: 19
This range should ideally only be 1, once we remove VIP orders!
Note: this is inclusive of VIP orders. Please verify this discrepancy!
------------------------------
MAXIMUM Order for Hospital: 21 for Hospital(s):
- Boston Healthcare for the Homeless 	Number of VIP orders: 14.0

MINIMUM Order for Hospital: 2 for Hospital(s):
- BIDMC - ICU West 	Number of VIP orders: 0.0
- BIDMC - ICU East 	Number of VIP orders: 0.0


In [266]:
print('Removing VIP Meals from the equation (VIP meals collapse to a single meal for each hospital):')
(hospital_meals_rollup[
    'total_orders_fulfilled'
] - hospital_meals_rollup[
    'vip_meals_count'
].transform(lambda x: max(0, x-1))).describe()

Removing VIP Meals from the equation (VIP meals collapse to a single meal for each hospital):


count    34.000000
mean      5.970588
std       3.911795
min       1.000000
25%       3.250000
50%       5.000000
75%       7.000000
max      17.000000
dtype: float64